In [128]:
import pandas as pd
import random
# pd.set_option('display.max_columns', 500)

In [129]:
columns = ['code', 'url', 'product_name', 'cholesterol_100g', 'sugars_100g', 'energy_100g']
food_data = pd.read_csv('en.openfoodfacts.org.products.csv', sep='\t', nrows=2000, usecols=columns)

In [130]:
food_data.fillna(food_data.median(), inplace=True)
#food_data.head()

In [131]:
# Create random user database
user_columns = ['user_id', 'food_id', 'food_time', 'rating']
user_data = pd.DataFrame(columns=user_columns)

for i in range(1000):
    for j in range(random.randrange(3, 30)): 
        random_foods = food_data.sample(n=3)
        breakfast = random_foods.iloc[0]
        lunch = random_foods.iloc[1]
        dinner = random_foods.iloc[2]

        user_data = user_data.append({
            'user_id': i,
            'food_id': breakfast['code'],
            'food_time': 'breakfast',
            'rating': random.choice([1,0])
        }, ignore_index = True)
        
        user_data = user_data.append({
            'user_id': i,
            'food_id': lunch['code'],
            'food_time': 'lunch',
            'rating': random.choice([1,0])
        }, ignore_index = True)
        
        user_data = user_data.append({
            'user_id': i,
            'food_id': dinner['code'],
            'food_time': 'dinner',
            'rating': random.choice([1,0])
        }, ignore_index = True)

user_data.head()

,user_id,food_id,food_time,rating
0,0,00073677,breakfast,1
1,0,0000630135899,lunch,0
2,0,00031271,dinner,1
3,0,00076739,breakfast,1
4,0,0000000030724,lunch,1


In [134]:
food_data.loc[food_data['code'] == '0000000030724']

,code,url,product_name,energy_100g,cholesterol_100g,sugars_100g
157,0000000030724,http://world-en.openfoodfacts.org/product/0000...,Organic Dark Red Kidney Bean,1272.0,0.0,2.17


In [123]:
from recsys import *

interactions = create_interaction_matrix(df = user_data,
                                         user_col = 'user_id',
                                         item_col = 'food_id',
                                         time_col = 
                                         rating_col = 'rating')

#interactions.head()

In [127]:
user_data.loc[user_data['user_id'] == 24]

,user_id,food_id,food_time,rating
624,24,281591,breakfast,1
625,24,790410058,lunch,0
626,24,901005005,dinner,0
627,24,433821500,breakfast,0
628,24,18524,lunch,0
629,24,80919704,dinner,0
630,24,1311,breakfast,1
631,24,80919704,lunch,0
632,24,8686,dinner,1
633,24,18517,breakfast,1


In [116]:
import pickle

f = open("int.pkl","wb")
pickle.dump(interactions,f)
f.close()

In [117]:
user_dict = create_user_dict(interactions=interactions)
food_dict = create_item_dict(df = food_data,
                               id_col = 'code',
                               name_col = 'product_name')

In [135]:
#export food/user dicts for use in backend

f = open("food_dict.pkl","wb")
pickle.dump(food_dict,f)
f.close()

f = open("food_data.pkl","wb")
pickle.dump(food_data,f)
f.close()

f = open("user_dict.pkl","wb")
pickle.dump(user_dict,f)
f.close()

In [119]:
mf_model = runMF(interactions = interactions,
                 n_components = 30,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

#export model
f = open("model.pkl","wb")
pickle.dump(mf_model,f)
f.close()

In [120]:
loaded_model = pickle.load(open("model.pkl", 'rb'))

rec = sample_recommendation_user(model = loaded_model,interactions = interactions,user_id = 21, user_dict = user_dict, item_dict = food_dict,threshold = 0,nrec_items = 10, show = False)

In [109]:
rec

[18568011,
 281591,
 1576,
 34470,
 34425,
 18927207,
 10207260,
 10166697,
 27262832,
 394710]